In [ ]:
import time
import scipy.special
import numpy as np
import matplotlib.pyplot as plt
from pyscf import gto, scf, fci, dft

# set the basis
basis = '6-31G'

# define molecule parameters: H2
bond_length = 0.702
geometry = 'H 0 0 {}; H 0 0 {}'
spin = 0

# build the molecule and set the verbosity level
mol = gto.M(atom    = geometry.format(-0.5 * bond_length, 0.5 * bond_length),
            basis   = basis,
            spin    = spin,
            verbose = 0)

# run the HF calculations
hf_calc = scf.UHF(mol)
res = hf_calc.scf()
print('{:32s}: {:.4f} Ha'.format('HF energy', res))

# run the FCI calculations
hf_calc = scf.RHF(mol) # we cheat a bit here because the pyscf CI solver only accepts RHF
hf_calc.kernel()
ci_calc = fci.FCI(mol, hf_calc.mo_coeff)
res = ci_calc.kernel()
res = res[0] # extract the ground state energy
print('{:32s}: {:.4f} Ha'.format('Full CI energy', res))

#run DFT calculations
for functional in ['LDA', 'PBE', 'HSE06', 'BLYP', 'B3LYP']:
    dft_calc = dft.KS(mol)
    dft_calc.xc = functional
    res = dft_calc.kernel()
    print('{:32s}: {:.4f} Ha'.format('{} energy'.format(functional), res))